In [20]:
merged = pd.merge(orders[['zipcode']], zipcounty[['zipcode', 'latitude', 'longitude']],
                  on='zipcode', how='inner')

merged = merged.drop_duplicates(keep='first')

bearings_list = []

for i in list(zip(merged['latitude'], merged['longitude'])):
    ptA = (37.0902, -95.7129)
    bearings_list.append(calculate_initial_compass_bearing(ptA, i))

merged['bearings'] = bearings_list

merged['polar'] = pd.cut(merged['bearings'], 16, labels=['N', 'NNE', 'NE', 'ENE',
                                                        'E', 'ESE', 'SE', 'SSE',
                                                        'S', 'SSW', 'SW', 'WSW',
                                                        'W', 'WNW', 'NW', 'NNW'])

orders['orderyear'] = pd.to_datetime(orders['orderdate']).dt.year

merged_1 = pd.merge(orders[['orderid', 'orderyear', 'campaignid', 'zipcode', 'totalprice']],
                    campaigns[['campaignid', 'channel']],
                    on='campaignid', how='inner')

merged_2 = pd.merge(merged_1, zipcounty[['zipcode', 'latitude', 'longitude']], on='zipcode')

z = merged_1.groupby(['zipcode', 'channel', 'orderyear'])['totalprice'].mean()

z.head()

zipcode  channel  orderyear
*        WEB      2013         156.00
                  2014          27.90
-        AD       2011          34.95
--       WEB      2015         199.90
.        WEB      2015          17.50
Name: totalprice, dtype: float64

In [21]:
z = z.unstack(level=-1)

z.head()

,orderyear,2009,2010,2011,2012,2013,2014,2015,2016
zipcode,channel,,,,,,,,
*,WEB,NaN,NaN,NaN,NaN,156.0,27.9,NaN,NaN
-,AD,NaN,NaN,34.95,NaN,NaN,NaN,NaN,NaN
--,WEB,NaN,NaN,NaN,NaN,NaN,NaN,199.9,NaN
.,WEB,NaN,NaN,NaN,NaN,NaN,NaN,17.5,22.95
.....,AD,NaN,NaN,NaN,37.61,NaN,NaN,NaN,NaN


In [22]:
z = z.unstack(level=-1)

z.head()

orderyear 2009                                                              \
channel     AD BULK CATALOG CONFERENCE EMAIL EMPLOYEE INSERT INTERNAL MAIL   
zipcode                                                                      
*          NaN  NaN     NaN        NaN   NaN      NaN    NaN      NaN  NaN   
-          NaN  NaN     NaN        NaN   NaN      NaN    NaN      NaN  NaN   
--         NaN  NaN     NaN        NaN   NaN      NaN    NaN      NaN  NaN   
.          NaN  NaN     NaN        NaN   NaN      NaN    NaN      NaN  NaN   
.....      NaN  NaN     NaN        NaN   NaN      NaN    NaN      NaN  NaN   

orderyear          ...       2016                                              \
channel   PARTNER  ... CONFERENCE EMAIL EMPLOYEE INSERT INTERNAL MAIL PARTNER   
zipcode            ...                                                          
*             NaN  ...        NaN   NaN      NaN    NaN      NaN  NaN     NaN   
-             NaN  ...        NaN   NaN      NaN    NaN      NaN  NaN     NaN   
--            NaN  ...        NaN   NaN      NaN    NaN      NaN  NaN     NaN   
.             NaN  ...        NaN   NaN      NaN    NaN      NaN  NaN     NaN   
.....         NaN  ...        NaN   NaN      NaN    NaN      NaN  NaN     NaN   

orderyear                         
channel   REFERRAL SURVEY    WEB  
zipcode                           
*              NaN    NaN    NaN  
-              NaN    NaN    NaN  
--             NaN    NaN    NaN  
.              NaN    NaN  22.95  
.....          NaN    NaN    NaN  

[5 rows x 104 columns]

In [23]:
z = z.unstack(level=-1)

z.head()

orderyear  channel  zipcode
2009       AD       *         NaN
                    -         NaN
                    --        NaN
                    .         NaN
                    .....     NaN
dtype: float64

In [24]:
z.fillna(0, inplace=True)



z.head(20)

orderyear  channel  zipcode
2009       AD       *            0.0
                    -            0.0
                    --           0.0
                    .            0.0
                    .....        0.0
                    0            0.0
                    0000         0.0
                    00000      900.0
                    00120        0.0
                    00157        0.0
                    00162        0.0
                    00177        0.0
                    00179        0.0
                    00185        0.0
                    00186        0.0
                    00193        0.0
                    00196        0.0
                    00198        0.0
                    0027         0.0
                    00646        0.0
dtype: float64

In [26]:
z['zipcode'] = z.index

In [31]:
type(z)

pandas.core.series.Series

In [32]:
z.reset_index(drop=True, inplace=False)

z.head()

orderyear  channel  zipcode
2009       AD       *          0
                    -          0
                    --         0
                    .          0
                    .....      0
dtype: object

In [10]:




final = pd.merge(z, merged[['zipcode', 'polar']], on='zipcode', how='inner')

mean_final = final.groupby('polar').mean()

mean_final = mean_final.reset_index()

mean_final.melt(id_vars=['polar'],
                var_name="Channel",
                value_name="Value")
a = mean_final.melt(id_vars=['polar'],
                    var_name="Channel",
                    value_name="Value")

,polar,Channel,Value
0,N,AD,29.970158
1,NNE,AD,27.170507
2,NE,AD,33.404761
3,ENE,AD,34.855027
4,E,AD,29.141724
5,ESE,AD,32.773886
6,SE,AD,21.645237
7,SSE,AD,35.406856
8,S,AD,26.855559
9,SSW,AD,43.356862
